# HITCON CTF 2019 Quals Writeups for Crypto challenges
## from _pwnPHOfun_ team

|Name|Total solves|Points|Solved during contest?|
|---|---|---|---|
|Lost modulus again|64|200|✓|
|Very simple Haskell|64|200|✓|
|Lost key again|10|334|&nbsp;|
|Not so hard RSA|6|371|✓|
|Randomly select a cat|3|421|&nbsp;|

## 1. Lost modulus again

-   Solves: _64_
-   Points: _200_
-   Status: _Done (during contest)_
-   Tags: _RSA_, _system of 2 equations of $p$ and $q$_


### 1.1. Challenge summary 

Find $p$ and $q$, given:
-  $E$: the public exponent
-  $D$: the private exponent
-  $I_p = p^{-1} \text{mod } q$ 
-  $I_q = q^{-1} \text{mod } p$.

### 1.2. Solution

The first equation:
$$
ED = K(p-1)(q-1) + 1
$$

The second equation:
$$
I_pp + I_qq = pq + 1
$$

Solve the two equations with SageMath symbolic computation:

In [1]:
var("ED, K, Ip, Iq, K, p, q")
solutions = solve([ED == K*(p-1)*(q-1) + 1, Ip*p + Iq*q == p*q + 1], p, q)
for solution in solutions:
    print solution[0], '\n'
    print solution[1], '\n'

p == -(ED*Iq - ((Ip + 2)*Iq - Iq^2 - 2)*K - Iq*(sqrt((Ip^2 + 2*(Ip - 2)*Iq + Iq^2 - 4*Ip + 4)*K^2 + ED^2 - 2*(((2*Ip - 1)*Iq - Ip)*ED - (2*Ip - 1)*Iq + Ip)*K - 2*ED + 1) + 1))/(((2*Ip - 1)*Iq - Ip)*K - ED + sqrt((Ip^2 + 2*(Ip - 2)*Iq + Iq^2 - 4*Ip + 4)*K^2 + ED^2 - 2*(((2*Ip - 1)*Iq - Ip)*ED - (2*Ip - 1)*Iq + Ip)*K - 2*ED + 1) + 1) 

q == -1/2*((Ip - Iq)*K - ED + sqrt((Ip^2 + 2*(Ip - 2)*Iq + Iq^2 - 4*Ip + 4)*K^2 + ED^2 - 2*(((2*Ip - 1)*Iq - Ip)*ED - (2*Ip - 1)*Iq + Ip)*K - 2*ED + 1) + 1)/((Iq - 1)*K) 

p == -(ED*Iq - ((Ip + 2)*Iq - Iq^2 - 2)*K + Iq*(sqrt((Ip^2 + 2*(Ip - 2)*Iq + Iq^2 - 4*Ip + 4)*K^2 + ED^2 - 2*(((2*Ip - 1)*Iq - Ip)*ED - (2*Ip - 1)*Iq + Ip)*K - 2*ED + 1) - 1))/(((2*Ip - 1)*Iq - Ip)*K - ED - sqrt((Ip^2 + 2*(Ip - 2)*Iq + Iq^2 - 4*Ip + 4)*K^2 + ED^2 - 2*(((2*Ip - 1)*Iq - Ip)*ED - (2*Ip - 1)*Iq + Ip)*K - 2*ED + 1) + 1) 

q == -1/2*((Ip - Iq)*K - ED - sqrt((Ip^2 + 2*(Ip - 2)*Iq + Iq^2 - 4*Ip + 4)*K^2 + ED^2 - 2*(((2*Ip - 1)*Iq - Ip)*ED - (2*Ip - 1)*Iq + Ip)*K - 2*ED + 1) + 1)

Brute-force `K` until `(Ip^2 + 2*(Ip - 2)*Iq + Iq^2 - 4*Ip + 4)*K^2 + ED^2 - 2*(((2*Ip - 1)*Iq - Ip)*ED - (2*Ip - 1)*Iq + Ip)*K - 2*ED + 1` is a perfect square:

In [2]:
E = 1048583
D = 20899585599499852848600179189763086698516108548228367107221738096450499101070075492197700491683249172909869748620431162381087017866603003080844372390109407618883775889949113518883655204495367156356586733638609604914325927159037673858380872827051492954190012228501796895529660404878822550757780926433386946425164501187561418082866346427628551763297010068329425460680225523270632454412376673863754258135691783420342075219153761633410012733450586771838248239221434791288928709490210661095249658730871114233033907339401132548352479119599592161475582267434069666373923164546185334225821332964035123667137917080001159691927
ED = E*D
Ip = 22886390627173202444468626406642274959028635116543626995297684671305848436910064602418012808595951325519844918478912090039470530649857775854959462500919029371215000179065185673136642143061689849338228110909931445119687113803523924040922470616407096745128917352037282612768345609735657018628096338779732460743
Iq = 138356012157150927033117814862941924437637775040379746970778376921933744927520585574595823734209547857047013402623714044512594300691782086053475259157899010363944831564630625623351267412232071416191142966170634950729938561841853176635423819365023039470901382901261884795304947251115006930995163847675576699331
for K in range(E, -1, -1):
    delta = (Ip^2 + 2*(Ip - 2)*Iq + Iq^2 - 4*Ip + 4)*K^2 + ED^2 - 2*(((2*Ip - 1)*Iq - Ip)*ED - (2*Ip - 1)*Iq + Ip)*K - 2*ED + 1
    if is_square(delta):
        break
print 'K =', K

K = 973605


Deduce $p$, $q$:

In [3]:
q = -1/2*((Ip - Iq)*K - ED - sqrt(delta) + 1)/((Iq - 1)*K)
if not q.is_integer():
    q = -1/2*((Ip - Iq)*K - ED + sqrt(delta) + 1)/((Iq - 1)*K)
    assert q.is_integer() 

p = (ED-1) / K / (q-1) + 1
assert p.is_integer()

print 'p = ', p
print 'q = ', q

p =  166564961597106229342966450443567005929416288372170308009700499465281616388542030734600089639466922805142577582894052957547174764252067983124558747593344882775630971023610755334859287415681252266825395627416912206349590353878868107848653100299011246746851490276537231636534462821659050996145878589643016881929
q =  135136928230019073146158752709151576119155564982754768027494878210491711363872928718225319693774548227271767324623087432404412585662574641211148315864508708036871556964386141364368797168619283425834644924573664613109609076319077176698754203574237779054129492453503018443013301394555677417140681949745410143477


Decrypt the flag:

In [4]:
from Crypto.Util.number import long_to_bytes
c = 0x32074de818f2feeb788e36d7d3ee09f0000381584a72b2fba0dcc9a2ebe5fd79cf2d6fd40c4dbfea27d3489704f2c1a30b17a783baa67229d02043c5bc9bdb995ae984d80a96bd79370ea2c356f39f85a12d16983598c1fb772f9183441fea5dfeb5b26455df75de18ce70a6a9e9dbc0a4ca434ba94cf4d1e5347395cf7aafa756c8a5bd6fd166bc30245a4bded28f5baac38d024042a166369f7515e8b0c479a1965b5988b350064648738f6585c0a0d1463bd536d11a105bb926b44236593b5c6c71ef5b132cd9c211e8ad9131aa53ffde88f5b0df18e7c45bcdb6244edcaa8d386196d25297c259fca3be37f0f2015f40cb5423a918c51383390dfd5a8703
print long_to_bytes(pow(c, D, p*q))

hitcon{1t_is_50_easy_t0_find_th3_modulus_back@@!!@!@!@@!}



## 2. Very simple Haskell

-   Solves: _64_
-   Points: _200_
-   Status: _Done (during contest)_
-   Tags: _Unpopular language_, _modular arithmetic_

### 2.1. Challenge summary 

A hash function $h$ which takes as input a binary string $m$ and ouputs an element of $Z_n$, works as follows:

1.  Cuts $m$ into 131-bit blocks $m_i$'s. Last block will be padded with zero bits. A block representing the length of $m$ is also appended.

<center><img src='2-1.png'></center>

2.  For each block $m_i$, computes $s_i = \prod_{j=0}^{130}p_j^{b_j}$ in which $p_0, p_1, ... p_{130}$ are the first 131 primes: $2, 3, 5, ... 739$.

3.  Outputs $\sum_{i=0}^ts_i^{2^i}$

![](2-2.png)

_The mission_: find flag, given `h("the flag is hitcon{??????}")` (`??????` are 6 unknown characters).

### 2.2. Solution

The first step: test the length of `m = "the flag is hitcon{??????}"`:

In [5]:
from bitstring import BitArray

m = BitArray(bytes="the flag is hitcon{\x00\x00\x00\x00\x00\x00}")
original_length = len(m)
print original_length 

208


If $m$ is padded, it will look like:

<center><img src='2-3.png' align='center'></center>

Observe that $h(m)$ would be equal to $h(m_\text{ without flag}) * h(m_\text{ with flag only})$:

<center><img src='2-4.png' align='center'></center>

Compute the hash value for $m_\text{without flag}$:

In [6]:
primes = primes_first_n(131)
n = 134896036104102133446208954973118530800743044711419303630456535295204304771800100892609593430702833309387082353959992161865438523195671760946142657809228938824313865760630832980160727407084204864544706387890655083179518455155520501821681606874346463698215916627632418223019328444607858743434475109717014763667

m = m + BitArray(length=(-original_length % 131)) + BitArray(length=131, uint=original_length)[::-1]

s2 = product(p^b for b,p in zip(m[:131], primes))
s1 = product(p^b for b,p in zip(m[131:131*2], primes))
s0 = product(p^b for b,p in zip(m[131*2:], primes))
hash_value_without_flag = s2^4*s1^2*s0 % n
print hash_value_without_flag

129105988525739869308153101831605950072860268575706582195774923614094296354415364173823406181109200888049609207238266506466864447780824680862439187440797565555486108716502098901182492654356397840996322893263870349262138909453630565384869193972124927953237311411285678188486737576555535085444384901167109670365


And then, $m_\text{with flag only}$:

In [7]:
hash_value = 84329776255618646348016649734028295037597157542985867506958273359305624184282146866144159754298613694885173220275408231387000884549683819822991588176788392625802461171856762214917805903544785532328453620624644896107723229373581460638987146506975123149045044762903664396325969329482406959546962473688947985096
hash_value_with_flag_only = hash_value * inverse_mod(hash_value_without_flag, n) % n
print hash_value_with_flag_only

3406218222930966554172275269328526576001581947668541896752967656582693660956578801


Finally, deduce the flag:

In [8]:
for p, _ in list(factor(sqrt(hash_value_with_flag_only))):
    m[131 + primes.index(p)] = 1
    
print m[:original_length].bytes

the flag is hitcon{v@!>A#}


## 3. Lost key again

-   Solves: _10_
-   Points: _334_
-   Status: _Done (after contest)_
-   Tags: _RSA_, _encryption oracle_, _weak n_

### 3.1. Challenge summary

At the beginning of each connection, the server prints out the flag in encrypted form, and allows us to encrypt arbitrary messages that:

1.  start with 0x583a20
2.  are at most 18-byte long

### 3.2. RSA revisited

<center><img src='3-1.png'></center>

### 3.3. Solution

To recover $n$, we need to find a set of messages such that:
-   Each message satisfies the 2 conditions above (starts with 0x583a20 and is at most 18-byte long)
-   There exists a relation of the form: $\prod_i m_i = \prod_j m_j$ among them, for example: $m_1m_2 = m_3m_4m_5$

If we could do that, we can then ask the server to encrypt the messages and obtain $\prod_i \text{encrypt}(m_i) - \prod_j \text{encrypt}(m_j)$, which is a multiple of the modulus. Applying the GCD algorithm to those multiples of $n$ should give us $n$.

Observe that `0x[583a20]0100 * 0x[583a20]02 == 0x[583a20]01 * 0x[583a20]0200` (both equal to `0x[583a20]01 * 0x[583a20]02 * 256`), we can now recover $n$ easily:

In [9]:
from socket import create_connection
s = create_connection(("localhost", 5555l))
f = s.makefile('r')
enc_flag = int(f.readline().split()[-1], 16)

def encrypt(msg):
    assert len(msg) <= 18 and msg[:3] == 'X: '
    s.sendall(msg[3:].encode('hex') +'\n')
    return int(f.readline().split()[-1], 16)

x = encrypt('X: x')
y = encrypt('X: y')
z = encrypt('X: z')

x0 = encrypt('X: x\x00')
y0 = encrypt('X: y\x00')
z0 = encrypt('X: z\x00')

multiples = [x*y0 - x0*y, x*z0 - x0*z, y*z0 - y0*z]
n = reduce(GCD, multiples, 0)
print 'n =', n

n = 28152737628466294873353447700677616804377761540447615032304834412268931104665382061141878570495440888771518997616518312198719994551237036466480942443879131169765243306374805214525362072592889691405243268672638788064054189918713974963485194898322382615752287071631796323864338560758158133372985410715951157


Now, to decrypt the flag, we need the private exponent $d$, which can be obtained by:
1.  factoring $n = pq$ and taking the inverse of $e$ mod $\phi(n)$, or
2.  solving discrete logarithm modulo $n$.

Since we don't know $e$ and how the author of the challenge generates $n$, <2> should be the way to go. This is only possible when $phi(n)$ is composed of small primes. Although it's a little bit guessing, the key idea is: **If $\phi(n)$ is not smooth, the challenge can NOT be solved!**

During the contest, _pwnPHOfun_ team members didn't realize that and failed to make any progress.

Knowing that $\phi(n)$ is smooth, $n$ can be easily factored with Pollard's p-1 algorithm:

In [10]:
a = 2
pi = 2
B = 100000
done = False
while True:
    for _ in range(RR(log(B, pi)) + 1):
        a = power_mod(a, pi, n)
        p = GCD(a-1, n)
        if p > 1:
            q = n/p
            done = True
            break
    pi = next_prime(pi)
    if done or pi > B:
        break

if done:
    print 'p =', p
    print 'q =', q

p = 531268630871904928125236420930762796930566248599562838123179520115291463168597060453850582450268863522872788705521479922595212649079603574353380342938159
q = 52991530070696473563320564293242344753975698734819856541454993888990555556689500359127445576561403828332510518908254263289997022658687697289264351266523


Check that $p-1$ and $q-1$ are both smooth:

In [11]:
print 'p-1 =', factor(p-1), '\n'
print 'q-1 =', factor(q-1)

p-1 = 2 * 269 * 1013 * 1997 * 2293 * 5477 * 6521 * 8521 * 9029 * 13729 * 15511 * 16333 * 18119 * 19963 * 24329 * 25589 * 26561 * 37889 * 38231 * 38557 * 38851 * 47881 * 49477 * 49547 * 49549 * 59009 * 63149 * 64067 * 66733 * 67807 * 70139 * 76543 * 78277 * 85621 * 85991 * 88289 

q-1 = 2 * 12157 * 13649 * 16633 * 21821 * 26863 * 28591 * 30703 * 35449 * 37159 * 52553 * 53527 * 55381 * 56963 * 59743 * 63311 * 66533 * 70067 * 71971 * 74527 * 77471 * 78919 * 80803 * 81173 * 82633 * 85531 * 87251 * 88589 * 89443 * 92243 * 95911 * 97651 * 99689


Solve the discrete logarithm problem to obtain $d$:

In [12]:
from Crypto.Util.number import long_to_bytes, bytes_to_long
Zn = Zmod(n)

order_x = order_from_multiple(Zn(x), (p-1)*(q-1), operation='*')
d0 = discrete_log(Zn(bytes_to_long('X: x')), Zn(x), ord=order_x)
print d0

2748764453029468715490471865512981104503568334538179947907460033180834741032686038914076807893036256348167409898773991268845429440917707803028762271222653474558811675645699768997645383556243231432537314294182698391886429863445585266694138147720853168418911122241669203982108182211754679102390434551295510


Now, $d$ is of the form: $d_0 + k.\text{order}(x); k \in \mathbb{Z}$. It's time to decrypt the flag:

In [13]:
i = 0
while True:
    tmp = long_to_bytes(pow(enc_flag, d0 + i*order_x, n))
    if tmp[:3] == 'X: ':
        print tmp
        break  
    i += 1

�E69Tܹd����m7pF]����3�x:�N�V�ۙ�^���^�
��X7%�W�hitcon{test_flag}


## 4. Not so hard RSA

-   Solves: _6_
-   Points: _371_
-   Status: _Done (during contest)_
-   Tags: _RSA_, _LLL_

### 4.1. Challenge summary

10 RSA public keys $(n_i, e_i)$, each $n_i$ is 1024-bit long, are given. They all share the same 465-bit private exponent $d$: $de_i = k_i\phi(n_i) + 1$ for $i = 0...9$. We need to recover $d$ to decrypt the flag.

### 4.2. Solution

To solve this challenge, we can model it as a Shortest vector problem in lattice. Currently we have:

$$
\begin{cases}
de_0 - k_0n_0 = k_0(-p_0 - q_0 + 1) + 1 \\
de_1 - k_1n_1 = k_1(-p_1 - q_1 + 1) + 1 \\
... \\
de_9 - k_9n_9 = k_9(-p_9 - q_9 + 1) + 1 \\
d.1 = d
\end{cases}
$$

Which can be expressed as a vector equation:
$$
d   \begin{bmatrix} e_0  \\ e_1  \\ ... \\ e_9  \\ 1 \end{bmatrix} \; + \;
k_0 \begin{bmatrix} -n_0 \\ 0    \\ ... \\ 0    \\ 0 \end{bmatrix} \; + \;
k_1 \begin{bmatrix} 0    \\ -n_1 \\ ... \\ 0    \\ 0 \end{bmatrix} \; + \;
... \; + \;
k_9 \begin{bmatrix} 0    \\ 0    \\ ... \\ -n_9 \\ 0 \end{bmatrix} \; = \;
\begin{bmatrix} k_0(-p_0 - q_0 + 1) + 1 \\ k_1(-p_1 - q_1 + 1) + 1 \\ ... \\ k_9(-p_9 - q_9 + 1) + 1 \\ d \\ \end{bmatrix}
$$

Perform some testing on self-generated data:

In [14]:
from Crypto.Util.number import getPrime
d_nbits = 463l
nsamples = 10
d = getPrime(d_nbits)

ns, es = [], []
for _ in range(nsamples):
    p, q = [getPrime(512l) for _ in range(2)]
    ns.append(p*q)
    es.append(inverse_mod(d, (p-1)*(q-1)))

A = [es + [2^512]]
for i in range(nsamples):
    v = [0] * (nsamples+1)
    v[i] = -ns[i] 
    A.append(v)

print abs(matrix(ZZ, A).LLL()[0][-1]/2^512)
print d

13099367240257886562683326258692783040851233923346570701030399385907024803830424710521768422217691215186634189023257737622427698309844818963
13099367240257886562683326258692783040851233923346570701030399385907024803830424710521768422217691215186634189023257737622427698309844818963


After some trials and errors, we can see that LLL works quite well when `d_nbits <= 463`. For `d_nbits == 465`, it need more samples to successfully recover `d`. In this situation, we should try to brute-force a few bits of `d`. For example, we let `d = dx*B + dy` for `B=2^460`, brute-force `dx` and let LLL find `dy`:

$$
d_y \begin{bmatrix} e_0  \\ e_1  \\ ... \\ e_9  \\ 1 \end{bmatrix} \; + \;
k_0 \begin{bmatrix} -n_0 \\ 0    \\ ... \\ 0    \\ 0 \end{bmatrix} \; + \;
k_1 \begin{bmatrix} 0    \\ -n_1 \\ ... \\ 0    \\ 0 \end{bmatrix} \; + \;
... \; + \;
k_9 \begin{bmatrix} 0    \\ 0    \\ ... \\ -n_9 \\ 0 \end{bmatrix} \; = \;
\begin{bmatrix} -d_xBe_0 \\ -d_xBe_1 \\ ... \\ -d_xBe_9 \\ 0   \\ \end{bmatrix} \; + \;
\begin{bmatrix} k_0(-p_0 - q_0 + 1) + 1 \\ k_1(-p_1 - q_1 + 1) + 1 \\ ... \\ k_9(-p_9 - q_9 + 1) + 1 \\ d_y \\ \end{bmatrix}
$$

Again, test this on self-generated data:

In [15]:
from cvp import solve_cvp2
d_nbits = 465l; nsamples = 10; bf_bits = 13
B = 2^(d_nbits - bf_bits)
d = getPrime(d_nbits)
dx, dy = d // B, d % B

ns, es = [], []
for _ in range(nsamples):
    p, q = [getPrime(512l) for _ in range(2)]
    ns.append(p*q)
    es.append(inverse_mod(d, (p-1)*(q-1)))
enc2 = power_mod(2, es[0], ns[0])
    
min_dy, max_dy = 0, B
min_d, max_d = dx*B + min_dy, dx*B + max_dy
min_k, max_k = 0, max_d
min_p, max_p = 2^511, 2^512
min_delta, max_delta = min_k*2*min_p, max_k*2*max_p
    
A = [es + [1]]
for i in range(nsamples):
    v = [0] * (nsamples+1)
    v[i] = -ns[i]
    A.append(v)
t = [-dx*(B*e) - (max_delta+min_delta)/2 for e in es] + [(max_dy+min_dy)/2]
    
print solve_cvp2(matrix(A), vector(t), [1/(max_delta-min_delta)]*nsamples + [1/(max_dy-min_dy)])[-1]
print dy

759564132885693916644098080888827304532477423782808819649552228016508667384537972026077144767082609436299706920855192782016751996208027
759564132885693916644098080888827304532477423782808819649552228016508667384537972026077144767082609436299706920855192782016751996208027


Now, work on real data:

In [16]:
with open('output') as f:
    d_nbits = eval(f.readline())
    necs = [eval(f.readline()) for _ in range(10)]
    ns, es, cs = [[x[i] for x in necs] for i in range(3)]

enc2 = power_mod(2, es[0], ns[0])
print 'Real data imported!'

Real data imported!


In [17]:
for dx in range(2^(bf_bits-1), 2^(bf_bits)):
    print bin(dx)
    min_dy, max_dy = 0, B
    min_d, max_d = dx*B + min_dy, dx*B + max_dy
    min_k, max_k = 0, max_d
    min_p, max_p = 2^511, 2^512
    min_delta, max_delta = min_k*2*min_p, max_k*2*max_p

    A = [es + [1]]
    for i in range(nsamples):
        v = [0] * (nsamples+1)
        v[i] = -ns[i]
        A.append(v)
    t = [-dx*(B*e) - (max_delta+min_delta)/2 for e in es] + [(max_dy+min_dy)/2]

    dy = solve_cvp2(matrix(A), vector(t), [2/(max_delta-min_delta)]*nsamples + [2/(max_dy-min_dy)])[-1]
    d = dx*B + dy
    if pow(enc2, d, ns[0]) == 2:
        print d
        break

0b1000000000000
0b1000000000001
0b1000000000010
0b1000000000011
0b1000000000100
0b1000000000101
0b1000000000110
0b1000000000111
0b1000000001000
0b1000000001001
0b1000000001010
0b1000000001011
0b1000000001100
0b1000000001101
0b1000000001110
0b1000000001111
0b1000000010000
0b1000000010001


KeyboardInterrupt: 

After a few minutes...

<center><img src='3-2.png' style='width: 200%;'></center>

Now, let's decrypt the flag and finish the challenge:

In [18]:
from Crypto.Util.number import long_to_bytes
d = 52734116413990333859776618028885273976613159615407987318262228643225868562829470128730568170502025104920857931843837285163945541735508099991
print long_to_bytes(pow(int(cs[0],16), d, ns[0]))

hitcon{recover_everything_by_amazing_LLL_algorithm!!}
}w�Hv��X��xH�Z'��o���W���G�������


## 5. Randomly select a cat

-   Solves: _3_
-   Points: _371_
-   Status: _Done (after contest)_
-   Tags: _RSA_, _LLL_, _small e_, _signing oracle_

### 5.1. Challenge summary

For each connection, the server generates a 2048-bit RSA public key $(n, e=3)$. $n$ is not known to us.

To sign a message $m$:

<center><img src='5-1.png' style='height: 70vh;'></center>

To encrypt a message m:

<center><img src='5-2.png'></center>

The server allows us to sign arbitrary messages, our task is to obtain a valid signature and the ciphertext of a message of the form `"meow*\n" + anything`.

### 5.2. Solution

Let's try to recover $n$ first. Let $s$ be a signature received from the server for an arbitrary message, we have $s = m^d \; \text{mod} \; n$ in which $m$ is of the form: `0x[fixed]00000000....000[compressed]`. Therefore, $c = s^3$ - `0x[fixed]00000000000000....000000` will be a good approximate to a multiple of $n$: $c = kn + \epsilon$.

With a lot samples of $c$, it's possible to recover $n$ using, again, LLL algorithm. For a pair ($c_i, c_j$), we have: $c_i = k_in + \epsilon_i$, $c_j = k_j*n + \epsilon_j$, then:

$$
c_ik_j - c_jk_i = \epsilon_ik_j - \epsilon_jk_i
$$

From that, we construct the following vector equation, which can be solved with LLL algorithm:

$$
k_0 \begin{bmatrix} 1    \\ c_1  \\ c_2  \\ ... \end{bmatrix} \; + \;
k_1 \begin{bmatrix} 0    \\ -c_0 \\ 0    \\ ... \end{bmatrix} \; + \;
k_2 \begin{bmatrix} 0    \\ 0    \\ -c_0 \\ ... \end{bmatrix} \; + \;
... \; = \;
\begin{bmatrix} k_0 \\ k_0\epsilon_1 - k_1\epsilon_0 \\ k_0\epsilon_2 - k_2\epsilon_0\\ ...\end{bmatrix}
$$

Now, let's collect some signatures from the server:

In [19]:
from Crypto.Util.number import bytes_to_long, long_to_bytes
from socket import create_connection
_s = create_connection(('localhost', 5556l))
_f = _s.makefile('r')

def get_sig(m):
    _f.readline()
    _s.sendall('meow~\n')
    _f.readline()
    _s.sendall(m +'\n')
    return eval(_f.readline())

FIXED = "\xCA\xFE\x12\x04"
PAD_OF_NULL = FIXED.ljust(255, '\x00')

nsamples = 20
Cs = [get_sig(str(i))^3 - bytes_to_long(PAD_OF_NULL) for _ in range(nsamples)]
print 'OK!'

OK!


Recover $n$ with LLL algorithm:

In [20]:
A = [[1] + Cs[1:]]
for i in range(1,20):
    v = [0]*20
    v[i] = -Cs[0]
    A.append(v)

k0 = abs(matrix(A).LLL()[0][0])
n = Cs[0]//k0
print 'n =', n

n = 26847544434668989774778912221335020354834663459123684471818532161674819851672627903855261920171959627805592800347280963830247260988743125256121328161834985146896156013199498269109946780406687322225209471558950181408560064911979755968357024588255018801202988618435470837096236486189752905975348238933558122183663091030800896818952282203831697878262249619357710771987737162455694492710787444129848805695653565519639390641351825168861918617625108349973767847827476397179228777039667658522576430788214290704432357265703715128970071912268205369249687973776454575171621071694436396354363844996699409100640863400212489102773


Now, since we know $n$ and $e$, we can encrypt any message we want. However, we still need to forge a valid signature for a message of the desired format, and here's the trick:

We don't need to find for an $s$ such that $s^3 \; \text{mod} \; n =$ `0x[fixed]0000000...00[compressed]`. `0x[fixed][compressed][junk]` is still acceptable and decompress to the same JSON object as the former!

Since, $e = 3$ is small, it's possible that we can find a small $s$ such that $s^3$ (without mod $n$) is of the form `0x[fixed][compressed][junk]`.

<center><img src='5-3.png'></center>

When the length of `[fixed][compressed]` is about 1/3 of `[fixed][compressed][junk]`, $s$ can be found easily by taking the cube root of `0x[fixed][compressed]0000000000...00`.

While the total length is 255 bytes, `[fixed]` is 4 bytes, it's required that the length of `[compressed]` is around or below 80 ~ 81 bytes. Unfortunately, normal Python Zlib compression can only achieve 87 ~ 88 bytes for the type of JSON string we're currently working with. However, there's an algorithm called _zopfli_ which is very good at minimizing the size of data after compressed:

In [21]:
import hashlib
from zopfli.zlib import compress

for i in range(10000000):
    msg = 'meow*\n' + str(i)
    hval = int(hashlib.sha256(msg).hexdigest(), 16)
    if '11111111' in str(hval):
        print 'Message:', repr(msg)
        print 'Hash value:', hval
        break

s = '{"nonce":"11111111","hash":' + str(hval) + '}'
compressed_data = compress(s)
print 'Length after compressed:', len(compressed_data)

Message: 'meow*\n348240'
Hash value: 111111110534911002116186458850090478486505830236380720697077991762214365037668
Length after compressed: 80


Looks good! Now, let's forge the signature for the message we have found:

In [22]:
t = ZZ(bytes_to_long((FIXED + compressed_data).ljust(255, '\x00')))
s = (t.nth_root(3, truncate_mode=True)[0]+1)
sig = long_to_bytes(s)
print long_to_bytes(s^3).startswith(FIXED + compressed_data)

True


It's time to finish the challenge:

In [23]:
padded_message = FIXED + msg.rjust(255 - 4, '\x00')
ciphertext = long_to_bytes(bytes_to_long(padded_message)^3 % n)
_s.sendall('meow!\n')
_f.readline()
_s.sendall(ciphertext.encode('hex') + '\n')
_f.readline()
_s.sendall(sig.encode('hex') +'\n')
while True:
    print _f.readline(),

meow meow meow?
#!/usr/bin/env ruby
# encoding: ASCII-8BIT

require 'base64'
require 'digest'
require 'json'
require 'openssl'
require 'securerandom'
require 'zlib'

Dir.chdir(File.dirname(__FILE__))

SIZE = 1024
L = SIZE / 4 - 1
CAFE = "\xCA\xFE\x12\x04"

class String
  def enhex
    self.unpack('H*')[0]
  end
end

def gg(msg)
  puts "\eMEOW! #{msg}\e"
  exit 1
end

def gen_key
  e = 3.to_bn
  p = OpenSSL::BN::generate_prime(SIZE, false)
  q = OpenSSL::BN::generate_prime(SIZE, false)
  n = p * q
  phi = (p - 1) * (q - 1)
  d = e.mod_inverse(phi)
  [e, d, n]
end

def H(m)
  Digest::SHA256.hexdigest(m).to_i(16).to_bn
end

def unpad(s)
  gg 'meow zzz' unless s.size == L && s[0, 4] == CAFE
  s[4..-1].gsub(/^\x00*/, '')
end

def pad(s)
  zero = L - 4 - s.size
  gg 'meooooooooooooooooow' unless zero >= 0
  CAFE + "\x00" * zero + s
end

def sign(m, d, n)
  h = H(m)
  nonce = SecureRandom.base64(6)
  obj = {hash: h.to_i, nonce: nonce}
  num = pad(Zlib.deflate(obj.to_json)).enhex.to_i(16).to_b

KeyboardInterrupt: 

# THANK YOU FOR YOUR ATTENTION